### 7.1.2 使用 FFT 获得功率频谱密度估计

此示例说明如何使用 periodogram 和 fft 函数获得非参数化功率谱密度 (PSD) 估计。这些不同用例说明针对偶数长度输入、归一化频率和以赫兹表示的频率以及单边和双边 PSD 估计，如何正确缩放 fft 的输出。所有用例都使用矩形窗。
#### 具有指定采样率的偶数长度输入
对于一个采样率为1 kHz的偶数长度信号，分别使用fft和periodogram获得其周期图。比较二者的结果。

创建一个含 N(0,1) 加性噪声的100 Hz正弦波信号。采样频率为 1 kHz。信号长度为1000个采样。

使用 fft 获取周期图。信号是偶数长度的实数值信号。由于信号是实数值信号，您只需要对正负频率之一进行功率估计。为了保持总功率不变，将同时在两组（正频率和负频率）中出现的所有频率乘以因子 2。零频率 (DC) 和奈奎斯特频率不会出现两次。绘制结果。

In [2]:
using TySignalProcessing
using TySystemIdentification
using TyPlot
using TyMath
fs = 1000
t = 0:1/fs:1-1/fs
x = cos.(2*pi*100*t) + randn(size(t))
N = length(x)
xdft = fft(x)
xdft = xdft[1:Int(N/2)+1]
psdx = (1/(fs*N)) * abs.(xdft).^2
psdx[2:end-1] = 2*psdx[2:end-1]
freq = 0:fs/length(x):fs/2
plot(freq,pow2db(psdx))
grid("on")
title("Periodogram Using FFT")
xlabel("Frequency (Hz)")
ylabel("Power/Frequency (dB/Hz)")



PyObject <objects.mw_text.CYlabel object at 0x00000144FFA1CC48>

计算并使用 periodogram 绘制周期图。二者的结果相同。

In [3]:
figure()
periodogram(x,rectwin(N),N,fs;plotfig=true)

In [4]:
mxerr = max(psdx'-periodogram(x,rectwin(N),N,fs))

DimensionMismatch: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(1), Base.OneTo(501)), b has dims (Base.OneTo(501), Base.OneTo(1)), mismatch at 1

#### 具有归一化频率的输入
通过 fft 为使用归一化频率的输入生成周期图。创建一个带 N(0,1) 加性噪声的正弦波信号。该正弦波的角频率为 π/4 弧度/采样点。


In [5]:
N = 1000
n = 0:N-1
x = cos.(pi/4*n) + randn(size(n))

1000-element Vector{Float64}:
  2.9140332373995688
  0.930136089154601
  0.9203975900341506
 -0.8897238287239757
 -3.2491863966483625
 -2.0732758628835466
 -0.8136813789207596
  0.6087140907541786
  1.2522543265461183
  0.37410229465628264
  ⋮
  0.6895035174212186
  2.830792704866372
  0.11242920074639551
 -0.3004630741913627
 -0.5206931330199897
 -1.7978192866040315
 -0.32732436419696054
  0.2841199823936156
 -0.47663970588408777

使用 fft 获取周期图。信号是偶数长度的实数值信号。由于信号是实数值信号，您只需要对正负频率之一进行功率估计。为了保持总功率不变，将同时在两组（正频率和负频率）中出现的所有频率乘以因子 2。零频率 (DC) 和奈奎斯特频率不会出现两次。绘制结果。

In [6]:
xdft = fft(x)
xdft = xdft[1:Int(N/2+1)]
psdx = (1/(2*pi*N)) * abs.(xdft).^2
psdx[2:end-1] = 2*psdx[2:end-1]
freq = 0:2*pi/N:pi
plot(freq/pi,pow2db(psdx))
grid("on")
title("Periodogram Using FFT")
xlabel("Normalized Frequency (*pi rad/sample)")
ylabel("Power/Frequency (dB/(rad/sample))")

PyObject <objects.mw_text.CYlabel object at 0x000001448DDCC108>

计算并使用 periodogram 绘制周期图。二者的结果相同。

In [7]:
figure()
periodogram(x,rectwin(N),N;plotfig=true)

In [8]:
mxerr = maximum(psdx-periodogram(x,rectwin(N),N))

1.4210854715202004e-14

#### 具有归一化频率的复数值输入
使用 fft 为具有归一化频率的复数值输入生成周期图。采用一个带 N(0,1) 复噪声的复指数信号，角频率为 π/4 弧度/采样点。

In [9]:
N = 1000
n = 0:N-1
x = exp.(im*pi/4*n) + vec([1 im]*randn(2,N)/sqrt(2))

1000-element Vector{ComplexF64}:
    0.5546253990004792 - 0.0013883387232103588im
    0.9527133102817857 + 0.6333405824437im
   -1.1053832639785177 + 1.5785813726765592im
   -1.2811843827079286 + 1.5696286509729014im
   -0.8528454329838838 - 0.524167576158056im
   -0.9698493886962565 - 1.4582393850068933im
    -0.505232344732834 + 0.024095526675772083im
    0.7590695503967465 + 0.7158739037520667im
    1.2205013964194453 + 0.6721822017550767im
 -0.054137961730305495 + 1.356443113649687im
                       ⋮
   0.47730648928327946 - 1.0091027687136112im
   0.33054218598941676 - 0.880649284255135im
    1.0887420117593862 + 0.8949341027154898im
    -1.825425813115274 + 0.899975182789041im
    -0.785821448904019 + 0.7852721574998837im
   -1.4561784224750007 + 0.4359646884865223im
   -0.8432185128753683 - 0.6814081338045912im
   0.34180897531306415 - 1.9397478800146337im
    0.3353005977614127 + 0.27719478759206473im

使用 fft 获得周期图。由于输入是复数值，此处求 [0,2π) 弧度/采样点区间内的周期图。绘制结果。

In [10]:
xdft = fft(x)
psdx = (1/(2*pi*N)) * abs.(xdft).^2
freq = 0:2*pi/N:2*pi-2*pi/N
plot(freq/pi,pow2db(psdx))
grid("on")
title("Periodogram Using FFT")
xlabel("Normalized Frequency (*pi rad/sample)")
ylabel("Power/Frequency (dB/(rad/sample))")

PyObject <objects.mw_text.CYlabel object at 0x0000014493969748>

计算并使用 periodogram 绘制周期图。二者的结果相同。

In [11]:
figure()
periodogram(x,rectwin(N),N,"twosided";plotfig=true)

In [12]:
mxerr = maximum(psdx-periodogram(x,rectwin(N),N,"twosided"))

5.684341886080802e-14